In [1]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
import numpy as np
import sys
import import_ipynb
import quickplay


Using TensorFlow backend.


importing Jupyter notebook from quickplay.ipynb


In [2]:
x=quickplay.Simulation()
print("1 "+str(x.getBuyState(x.p1)))
print("2 "+str(x.getBuyState(x.p2)))
wins = [0]*10
for i in range(len(wins)):
    j=0
    while True:
        j+=1
        new_s, reward, end = x.makeDecision(x.p1,1)
        if end:
            break
        new_s, reward, end = x.makeDecision(x.p2,1)
        if end:
            break
    print(x.p1.getVP())
    print(x.p2.getVP())
    if x.p1.getVP() > x.p2.getVP():
        print(str(x.p1)+" wins!")
        wins[i]=1
    else:
        print(str(x.p2)+" wins!")
        wins[i]=2
    x=quickplay.Simulation()
print(str(wins))

1 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 3, 0, 0, 1, 2]
2 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 3, 0, 0, 1, 5]
1 p1: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 4, 0, 0, 1, 5]
1 p2: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 7, 0, 0, 3, 0, 0, 1, 2]
2 p1: [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 4, 0, 0, 1, 4]
2 p2: [0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 7, 0, 0, 3, 0, 0, 1, 2]
3 p1: [0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 7, 0, 0, 4, 0, 0, 1, 3]
3 p2: [0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 7, 0, 0, 3, 0, 0, 1, 5]
4 p1: [1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 7, 0, 0, 4, 0, 0, 1, 4]
4 p2: [0, 0, 0, 1, 2, 1, 0, 0, 1, 0, 7, 0, 0, 3, 0, 0, 1, 1]
5 p1: [1, 0, 2, 0, 0, 0, 0, 1, 0, 0, 7, 0, 0, 4, 0, 0, 1, 4]
5 p2: [0, 0, 0, 1, 2, 1, 0, 0, 1, 0, 7, 1, 0, 3, 0, 0, 1, 4]
6 p1: [1, 0, 2, 0, 0, 0, 0, 1, 0, 1, 7, 1, 0, 4, 0, 0, 1, 1]
6 p2: [0, 0, 0, 1, 2, 1, 1, 0, 1, 0, 7, 1, 0, 3, 0, 0, 1, 3]
7 p1: [1, 0, 2, 0, 0, 0, 0, 1, 0, 1, 8, 1, 0, 4, 0, 0, 1, 2]
7 p2: [0, 0, 0, 1, 3, 1, 1, 0, 2, 0, 7, 1, 0, 3, 0, 0, 1, 0]
8 p1: [1, 0, 2, 0, 1, 0, 0, 1, 0

In [53]:
#print(wins.count(1))
#print(wins.count(2))
print(x.shop.cards)
print(len(list(filter(lambda y: x.shop.cards[y]==0 , x.shop.cards)))>=3 or x.shop.cards["Province"]==0 )
print(x.shop.checkEnd())

{'Cellar': 0, 'Market': 4, 'Militia': 3, 'Mine': 10, 'Moat': 0, 'Remodel': 0, 'Smithy': 3, 'Village': 0, 'Woodcutter': 4, 'Workshop': 0, 'Copper': 17, 'Silver': 33, 'Gold': 26, 'Estate': 0, 'Dutchy': 7, 'Province': 6}
True
False


In [18]:
model = Sequential()
model.add(Dense(10, activation='sigmoid', input_shape=(18,)))
model.add(Dense(2, activation='linear'))
model.compile(loss='mse', optimizer='adam', metrics=['mae'])

In [19]:
y = 0.95
eps = 0.45
decay_factor = 0.999
r_avg_list = []
num_episodes = 50
for i in range(num_episodes):
    s = env.reset()
    eps *= decay_factor
    if i % 5 == 0:
        print("Episode {} of {}".format(i + 1, num_episodes))
    done = False
    r_sum = 0
    while not done:
        if np.random.random() < eps:
            a = np.random.randint(0, 2)
        else:
            a = np.argmax(model.predict(np.identity(5)[s:s + 1]))
        new_s, r, done, _ = env.step(a)
        target = r + y * np.max(model.predict(np.identity(5)[new_s:new_s + 1]))
        target_vec = model.predict(np.identity(5)[s:s + 1])[0]
        target_vec[a] = target
        model.fit(np.identity(5)[s:s + 1], target_vec.reshape(-1, 2), epochs=1, verbose=0)
        s = new_s
        r_sum += r
    r_avg_list.append(r_sum / 1000)

NameError: name 'env' is not defined